In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch 

# Model params in additional folder for additional model
# new_model = BartForConditionalGeneration.from_pretrained("./additional/")

# Model params in original folder for original model
# new_model = BartForConditionalGeneration.from_pretrained("./original")

new_model = BartForConditionalGeneration.from_pretrained("./test")

In [11]:
def bart_correct(tokenizer, model, text: str, max_length: int = 128):
    import numpy as np
    text = "< " + text
    inputs = tokenizer(text, return_tensors='pt')

    input_ids = inputs['input_ids'][:, :max_length]
    attention_mask = inputs['attention_mask'][:, :max_length]

    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits

        predicted_token_indexes = torch.argmax(logits, dim=-1)

        decode_tokens = tokenizer.decode(predicted_token_indexes[0], skip_special_tokens=True)

        decode_tokens = decode_tokens.strip()

    return decode_tokens

In [12]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
bart_correct(tokenizer, new_model, "He are a nice person.", 128)

'He is a nice person.'

In [13]:
f = open('../ABCN_dev.src')
orig_sentences = f.readlines()

print(len(orig_sentences))

4384


In [14]:
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
import spacy 

nlp = spacy.load("en_core_web_sm") 
f = open("bart_prefix_original_data.txt", 'w')
def generate_result():
    for i in tqdm(range(len(orig_sentences))):
        pred = bart_correct(tokenizer, new_model, orig_sentences[i], 128)
        tokens = nlp(pred)
        for token in tokens:
            f.write(str(token) + " ")
        
        f.write("\n")

generate_result()
f.close()

100%|██████████| 4384/4384 [08:09<00:00,  8.96it/s]
